<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/%E7%8E%89%E5%B1%B1%E9%8A%80%E8%A1%8CAI%E6%AF%94%E8%B3%BD_%E7%89%B9%E5%BE%B5%E5%89%B5%E5%BB%BA%E5%B7%A5%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 中文map

In [0]:
中文map = {'bacno':'歸戶帳號','txkey':'交易序號','locdt':'授權日期','loctm':'授權時間','cano':'交易卡號',
         'contp':'交易類別','etymd':'交易型態','mchno':'特店代號','acqic':'收單行代碼','mcc':'MCC_CODE',
         'conam':'交易金額-台幣(經過轉換)','ecfg':'網路交易註記','insfg':'分期交易註記','iterm':'分期期數',
         'stocn':'消費地國別','scity':'消費城市','stscd':'狀態碼','ovrlt':'超額註記碼','flbmk':'Fallback註記',
         'hcefg':'支付型態','csmcu':'消費地幣別','flg_3dsmk':'3DS交易註記','fraud_ind':'盜刷註記'}

# train data

In [3]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_預處理完.csv",index_col=0)
print(train.shape)
train.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1521787, 23)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,fraud_ind
0,6881,113261,38038,513.80,5,0,0,0,0,0,5,0,0,33,172652.0,457,59333,0,0,102,0,516056,0
1,0,134508,45725,465.62,5,0,0,2,0,0,0,0,0,9,105114.0,451,0,0,5817,102,0,4376,0
2,6881,15408,188328,513.80,5,0,0,0,0,0,5,0,0,6,152458.0,457,59333,0,0,102,0,483434,0
3,6716,157159,29967,1016.11,5,62,0,5,0,0,5,0,0,5,172946.0,247,50436,0,3281,102,0,1407164,0
4,5975,105985,81305,713.66,5,62,0,4,0,0,5,0,0,6,182129.0,263,93775,0,5817,102,0,1051004,0


# test data

In [4]:
test = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_預處理完.csv",index_col=0)
print(test.shape)
test.head()

(421665, 22)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,163188,116799,513.8,5,0,0,0,0,0,5,0,0,102,215328.0,457,59360,0,0,102,0,592489
1,6881,163188,116799,513.8,5,0,0,0,0,0,5,0,0,102,222007.0,457,59360,0,0,102,0,592452
2,6881,163188,116799,513.8,5,0,0,0,0,0,5,0,0,100,170013.0,457,59360,0,0,102,0,590212
3,6881,163188,116799,513.8,5,0,0,0,0,0,5,0,0,100,165914.0,457,59360,0,0,102,0,590209
4,6881,163188,116799,513.8,5,0,0,0,0,0,5,0,0,102,215311.0,457,59360,0,0,102,0,592488


# 1.P

In [5]:
p = {}
for col in train.drop('fraud_ind',axis=1).columns:
  交集 = set(train[col].unique())&set(test[col].unique())
  測試資料 = set(test[col].unique())
  p[col] = len(交集)/len(測試資料)
pd.DataFrame(p,index=['p']).T.sort_values(by='p',ascending=False)

,p
insfg,1.000000
stscd,1.000000
contp,1.000000
ecfg,1.000000
etymd,1.000000
flbmk,1.000000
flg_3dsmk,1.000000
ovrlt,1.000000
iterm,1.000000
loctm,0.990267


# 在測試資料上scity的百分比只有0.66,這裡用scity map stocn的方式來建立新特徵

In [6]:
import numpy as np

fulldata = train.append(test)

fulldata['scity_map_stocn'] = np.nan

map_dict = {}
for value in fulldata['scity'].unique():
  map_dict[value] = fulldata.loc[fulldata['scity'] == value,'stocn'].mode().values[0].astype(int)
fulldata['scity_map_stocn'] = fulldata['scity'].map(map_dict)
print(fulldata.drop('fraud_ind',axis=1).isnull().sum().sum())
fulldata.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


0


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,scity_map_stocn
0,6881,113261,38038,513.80,5,0,0,0,0,0,0.0,5,0,0,33,172652.0,457,59333,0,0,102,0,516056,102
1,0,134508,45725,465.62,5,0,0,2,0,0,0.0,0,0,0,9,105114.0,451,0,0,5817,102,0,4376,102
2,6881,15408,188328,513.80,5,0,0,0,0,0,0.0,5,0,0,6,152458.0,457,59333,0,0,102,0,483434,102
3,6716,157159,29967,1016.11,5,62,0,5,0,0,0.0,5,0,0,5,172946.0,247,50436,0,3281,102,0,1407164,102
4,5975,105985,81305,713.66,5,62,0,4,0,0,0.0,5,0,0,6,182129.0,263,93775,0,5817,102,0,1051004,102


In [7]:
train = fulldata.loc[fulldata['fraud_ind'].notnull(),:]
test = fulldata.loc[fulldata['fraud_ind'].isnull(),:]
print(train.shape)
print(test.shape)

(1521787, 24)
(421665, 24)


In [8]:
train.isnull().sum().sum()

0

In [9]:
test.isnull().sum().sum()

421665

# save

In [10]:
train.to_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_特徵工程完.csv")
test.to_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_特徵工程完.csv")
print('save done!')

save done!
